In [136]:
import graphlab as gl
!ls ProfMachanavajjhala/Amazon-GoogleProducts

Amazon.csv                              Amzon_GoogleProducts_perfectMapping.csv GoogleProducts.csv


In [137]:
amazon = gl.SFrame.read_csv("ProfMachanavajjhala/Amazon-GoogleProducts/Amazon.csv")

PROGRESS: Finished parsing file /Users/ricardo/Documents/GuillermoGaps/spark-1.3.1/ipython_notebooks/ProfMachanavajjhala/Amazon-GoogleProducts/Amazon.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.094012 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,str,str,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/ricardo/Documents/GuillermoGaps/spark-1.3.1/ipython_notebooks/ProfMachanavajjhala/Amazon-GoogleProducts/Amazon.csv
PROGRESS: Parsing completed. Parsed 1363 lines in 0.063859 secs.


In [14]:
#amazon["name"] = amazon["title"]
amazon.head()
amazon.column_names()
amazon = amazon[['id', 'name','description', 'manufacturer', 'price']]
amazon.head()

id,name,description,manufacturer,price
b000jz4hqo,clickart 950 000 -premier image pack (dvd- ...,,broderbund,0.0
b0006zf55o,ca international -arcserve lap/desktop oem ...,oem arcserve backup v11.1win 30u for laptops and ...,computer associates,0.0
b00004tkvy,noah's ark activitycenter (jewel case ages ...,,victory multimedia,0.0
b000g80lqo,peachtree by sage premiumaccounting for nonpro ...,peachtree premiumaccounting for nonpro ...,sage software,599.99
b0006se5bq,singing coach unlimited,singing coach unlimited -electronic learning ...,carry-a-tune technologies,99.99
b000ehpzv8,emc retrospect 7.5 diskto disk windows ...,emc retrospect 7.5 diskto diskcromwindows ...,dantz,0.0
b00021xhzw,adobe after effectsprofessional 6.5 upgrade ...,upgrade only;installation of after ...,adobe,499.99
b000gzwjgc,acad upgrade dragonnaturallyspeaking pro ...,- marketing information:dragon naturallyspeak ...,nuance academic,399.54
b0000dbykm,mia's math adventure:just in time ...,in mia's math adventure:just in time children ...,kutoka,19.99
b00029bqa2,disney's 1st & 2nd gradebundle (pixar 1st grade ...,disney's 1st & 2nd gradebundle will help your ...,disney,14.99


In [8]:
google = gl.SFrame.read_csv("ProfMachanavajjhala/Amazon-GoogleProducts/GoogleProducts.csv")

PROGRESS: Finished parsing file /Users/ricardo/Documents/GuillermoGaps/spark-1.3.1/ipython_notebooks/ProfMachanavajjhala/Amazon-GoogleProducts/GoogleProducts.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.029119 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,str,str,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/ricardo/Documents/GuillermoGaps/spark-1.3.1/ipython_notebooks/ProfMachanavajjhala/Amazon-GoogleProducts/GoogleProducts.csv
PROGRESS: Parsing completed. Parsed 3226 lines in 0.030008 secs.


In [9]:
google.head()

id,name,description,manufacturer
http://www.google.com/base/feeds/snippets/1112 ...,learning quickbooks 2007,learning quickbooks 2007,intuit
http://www.google.com/base/feeds/snippets/1153 ...,superstart! fun withreading & writing! ...,fun with reading &writing! is designed to ...,
http://www.google.com/base/feeds/snippets/1134 ...,qb pos 6.0 basic software,qb pos 6.0 basic retailmngmt software. for ...,intuit
http://www.google.com/base/feeds/snippets/1204 ...,math missions: theamazing arcade adventure ...,save spectacle city bydisrupting randall ...,
http://www.google.com/base/feeds/snippets/1224 ...,production prem cs3 macupgrad ...,adobe cs3 productionpremium mac upgrade from ...,adobe software
http://www.google.com/base/feeds/snippets/1231 ...,video studio 11 plus,corel video studio 11plus is consumer video ...,corel corporation
http://www.google.com/base/feeds/snippets/1336 ...,edius pro 4,whether you are workingwith standard definition ...,canopus/grass valley
http://www.google.com/base/feeds/snippets/1283 ...,qb pos 6.0 pro multistore sw ...,qb pos 6.0 pro multistoreretail mngmt software. ...,intuit
http://www.google.com/base/feeds/snippets/1294 ...,qb credit card kit 3.0,quickbooks credit cardkit v3.0 (2007) ...,intuit
http://www.google.com/base/feeds/snippets/1299 ...,vegas movie studiodvd+8plat ...,sony media software vegasmovie studio +dvd 8 ...,sony creative software


In [135]:
dist = [[('name','description',), 'jaccard', 3]]
#        [('price',), 'manhattan', 0.3]]
#        [('manufacturer',), 'jaccard', 1]]
#dist_spec = [[('X1', 'X2'), 'euclidean', 2],
#              [('species',), 'levenshtein', 0.3]]
model = gl.nearest_neighbor_deduplication.create({'amazon':amazon,'google':google}, row_label='id', grouping_features=['manufacturer'], distance=dist)

[INFO] Constructing groups of records that match exactly on the 'grouping_features'.
[INFO] Processing 1 records in match group: 1/381
[INFO] Building the similarity graph....
[INFO] Finding duplicate records in the similarity graph....
[INFO] Processing 1 records in match group: 2/381
[INFO] Building the similarity graph....
[INFO] Finding duplicate records in the similarity graph....
[INFO] Processing 1 records in match group: 3/381
[INFO] Building the similarity graph....
[INFO] Finding duplicate records in the similarity graph....
[INFO] Processing 3 records in match group: 4/381
[INFO] Building the similarity graph....
[INFO] Finding duplicate records in the similarity graph....
[INFO] Processing 1 records in match group: 5/381
[INFO] Building the similarity graph....
[INFO] Finding duplicate records in the similarity graph....
[INFO] Processing 4 records in match group: 6/381
[INFO] Building the similarity graph....
[INFO] Finding duplicate records in the similarity graph....
[IN

PROGRESS: Starting to query
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1            | 1       | 100         | 1.019ms      |
PROGRESS: | Done         |         | 100         | 1.287ms      |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: +-----------------------------+
PROGRESS: | Number of components merged |
PROGRESS: +-----------------------------+
PROGRESS: | 0                           |
PROGRESS: +-----------------------------+
PROGRESS: Starting to query
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1            | 1       | 100         | 1.098ms      |
PROGRESS: | Done         |         | 100         | 1.459ms      |
PROGRESS

In [110]:
model.summary()

Class                               : NearestNeighborDeduplication

Schema
------
Number of input datasets            : 2
Number of feature columns           : 3
Number of neighbors per point (k)   : 2
Max distance to a neighbor (radius) : None
Number of entities                  : 1152
Total training time (seconds)       : 14.492

Training
--------
Total training time (seconds)       : 14.492

Accessible fields                   : 
   entities                         : Consolidated input records plus entity labels.


In [124]:
m1=model
entities1 = m1['entities']
# find the dupe IDs
import graphlab.aggregate as agg

entity_counts1 = m1['entities'].groupby('__entity', agg.COUNT)
dupe_entities1 = entity_counts1[entity_counts1['Count'] > 1]['__entity']
# print the results for the dupes
dupes1 = m1['entities'].filter_by(dupe_entities1, '__entity')
#dupes1.print_rows(40, max_row_width=100, max_column_width=50)
dupes1.column_names()

['__sframe', 'id', '__entity', 'description', 'manufacturer', 'name']

In [125]:
dupes1pd = dupes1.to_dataframe()

In [126]:
grouped  = dupes1pd.groupby(['__entity',"__sframe"]).size()

In [127]:
import pandas as pd
nat = pd.DataFrame(grouped.reset_index())


In [128]:
nat.columns = ["__entity","__sframe", "counts"]
nat.head(20)

,__entity,__sframe,counts
0,0,google,2
1,1,amazon,1
2,1,google,6
3,2,google,2
4,3,amazon,1
5,3,google,3
6,4,google,2
7,5,google,4
8,6,google,3
9,7,google,2


In [129]:
dupes = nat[nat.counts == 1]

In [130]:
len(dupes)

514

In [133]:
(1300.0-514.0)/len(nat[ nat.counts > 1])

0.7011596788581623

In [92]:
dupes1[['__sframe','__entity', 'name', 'manufacturer']]

__sframe,__entity,name,manufacturer
amazon,5,quicken 2007 basic,intuit
amazon,5,quicken 2007 for mac,intuit
google,10,punch software 41100 -punch! home design ...,
google,10,punch software 42100 -punch! home design ...,
amazon,12,red hat entprse linux as3 amd ( rhf0211us ) ...,red hat software
google,12,rhf0211us red hatenterprise linux as ...,
amazon,13,myinvoices & estimatesdeluxe ...,avanquest software
google,13,myinvoices & estimatesdeluxe ...,
google,17,myob accountedge 2007,
google,17,myob premier accountingfor small business ...,


In [86]:
dupes1.filter_by('b00004tkvy','id')

__sframe,id,__entity,description,manufacturer,name
amazon,b00004tkvy,475,,victory multimedia,noah's ark activitycenter (jewel case ages ...


In [88]:
dupes1.filter_by(475,'__entity')['id']

dtype: str
Rows: 2
['http://www.google.com/base/feeds/snippets/18441110047404795849', 'b00004tkvy']

In [76]:
google.filter_by('http://www.google.com/base/feeds/snippets/18434262004669557918', 'id')

id,name,description,manufacturer,price
http://www.google.com/base/feeds/snippets/1843 ...,webroot software inc -31250 - spy sweeper ...,a sypware infection is nolonger just a matter of ...,,24.81


In [47]:
perfect = gl.SFrame.read_csv("ProfMachanavajjhala/Amazon-GoogleProducts/Amzon_GoogleProducts_perfectMapping.csv")

PROGRESS: Finished parsing file /Users/ricardo/Documents/GuillermoGaps/spark-1.3.1/ipython_notebooks/ProfMachanavajjhala/Amazon-GoogleProducts/Amzon_GoogleProducts_perfectMapping.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.028556 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/ricardo/Documents/GuillermoGaps/spark-1.3.1/ipython_notebooks/ProfMachanavajjhala/Amazon-GoogleProducts/Amzon_GoogleProducts_perfectMapping.csv
PROGRESS: Parsing completed. Parsed 1300 lines in 0.013374 secs.


In [58]:
perfect.head()['idGoogleBase']

dtype: str
Rows: 10
['http://www.google.com/base/feeds/snippets/18441480711193821750', 'http://www.google.com/base/feeds/snippets/18441110047404795849', 'http://www.google.com/base/feeds/snippets/18441188461196475272', 'http://www.google.com/base/feeds/snippets/18428750969726461849', 'http://www.google.com/base/feeds/snippets/18430621475529168165', 'http://www.google.com/base/feeds/snippets/18434163358494173665', 'http://www.google.com/base/feeds/snippets/18363672170449359273', 'http://www.google.com/base/feeds/snippets/18431790997623472871', 'http://www.google.com/base/feeds/snippets/18434733520229761009', 'http://www.google.com/base/feeds/snippets/18434262004669557918']

In [123]:
len(set(perfect["idAmazon"]))

1113

In [134]:
len(set(perfect["idGoogleBase"]))

1291

In [122]:
perfect

idAmazon,idGoogleBase
b000jz4hqo,http://www.google.com/base/feeds/snippets/1844 ...
b00004tkvy,http://www.google.com/base/feeds/snippets/1844 ...
b000g80lqo,http://www.google.com/base/feeds/snippets/1844 ...
b0006se5bq,http://www.google.com/base/feeds/snippets/1842 ...
b00021xhzw,http://www.google.com/base/feeds/snippets/1843 ...
b0000dbykm,http://www.google.com/base/feeds/snippets/1843 ...
b0000dbykm,http://www.google.com/base/feeds/snippets/1836 ...
b00029bqa2,http://www.google.com/base/feeds/snippets/1843 ...
b0007prnjo,http://www.google.com/base/feeds/snippets/1843 ...
b000i82j80,http://www.google.com/base/feeds/snippets/1843 ...
